In [1]:
import sys
sys.path.append(r'C:\Codes\01_Personal\08_Cufuture\HelpDesk-AI\.venv\Lib\site-packages')
sys.path.append(r'C:\Codes\01_Personal\08_Cufuture\HelpDesk-AI')

In [56]:
import requests

url = "http://127.0.0.1:8000/register-complaints/"

payload = {
    "name": "Alice Smith",
    "mobile_number": "4157463210",
    "complaints": { 
        "complaint_details": "Internet connectivity issues since morning.",
    }
}


response = requests.post(url, json=payload)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'message': 'New complaint added', 'complaint_id': 'e3a36aaf-0d9e-4c2c-b2e0-967587ea37ad'}


In [61]:
import requests

url = "http://127.0.0.1:8000/status/"

payload = {
        "mobile_number": "4157463210",
        "complaint_id": 'e3a36aaf-0d9e-4c2c-b2e0-967587ea37ad',
    }


response = requests.post(url, json=payload)

print("Status Code:", response.status_code)
print("Response JSON:", response.json())

Status Code: 200
Response JSON: {'complaint_id': 'e3a36aaf-0d9e-4c2c-b2e0-967587ea37ad', 'status': 'In Progress', 'details': 'Internet connectivity issues since morning.', 'created_at': '2025-07-04T13:36:24.519000'}
